In [1]:
import pynq
import numpy as np
from pynq import Overlay, allocate
import pandas as pd
import time
%matplotlib inline

In [2]:
rails = pynq.get_rails()
rail_name = 'PSINT_FP'

In [3]:
def get_recorder():
    return pynq.DataRecorder(rails[rail_name].power)

In [4]:
# set cpu frequency to default value
!cpufreq-set -f 1200000
!cpufreq-info | grep " current CPU frequency"

  current CPU frequency is 1.20 GHz (asserted by call to hardware).
  current CPU frequency is 1.20 GHz (asserted by call to hardware).
  current CPU frequency is 1.20 GHz (asserted by call to hardware).
  current CPU frequency is 1.20 GHz (asserted by call to hardware).


In [5]:
!pwd

/home/xilinx/jupyter_notebooks


In [6]:
%cd ../test

/home/xilinx/test


In [7]:
!ls

design_1_ap_fixed.bit	design_1_fine_tune.hwh	test.csv
design_1_ap_fixed.hwh	inference_ap_fixed.py	test.py
design_1_fine_tune.bit	inference.py		venv


In [8]:
start_load = time.time()
overlay = Overlay('design_1_fine_tune.bit')
dma = overlay.axi_dma_0
print(f"Overlay loading time: {time.time() - start_load} s")

Overlay loading time: 0.6794521808624268 s


In [9]:
input_buffer = allocate(shape=(36,), dtype=np.float32)
output_buffer = allocate(shape=(1,), dtype=np.float32)

In [10]:
TEST_DATA = 'test.csv'

In [11]:
test_df = pd.read_csv(TEST_DATA)

test_df = test_df[~test_df['Activity'].isin(['STANDING', 'WALKING'])]
features = [
    'tBodyAcc-mean()-X', 'tBodyAcc-mean()-Y', 'tBodyAcc-mean()-Z', 'tBodyAcc-std()-X',
    'tBodyAcc-std()-Y', 'tBodyAcc-std()-Z', 'tBodyAcc-max()-X', 'tBodyAcc-max()-Y',
    'tBodyAcc-max()-Z', 'tBodyAcc-min()-X', 'tBodyAcc-min()-Y', 'tBodyAcc-min()-Z',
    
    'tBodyGyro-mean()-X', 'tBodyGyro-mean()-Y', 'tBodyGyro-mean()-Z', 'tBodyGyro-std()-X',
    'tBodyGyro-std()-Y', 'tBodyGyro-std()-Z', 'tBodyGyro-max()-X', 'tBodyGyro-max()-Y',
    'tBodyGyro-max()-Z', 'tBodyGyro-min()-X', 'tBodyGyro-min()-Y', 'tBodyGyro-min()-Z',
    
    'tGravityAcc-std()-X', 'tGravityAcc-std()-Y', 'tGravityAcc-std()-Z', 'tGravityAcc-mad()-X',
    'tGravityAcc-mad()-Y', 'tGravityAcc-mad()-Z', 'tGravityAcc-max()-X', 'tGravityAcc-max()-Y',
    'tGravityAcc-max()-Z', 'tGravityAcc-min()-X', 'tGravityAcc-min()-Y', 'tGravityAcc-min()-Z',
    
    'Activity'
]

test_df = test_df[[f for f in features]]

X_test = test_df.iloc[:, :-1]
y_test = test_df[['Activity']]

In [12]:
y_test_transformed = []

for i, row in y_test.iterrows():
    activity = row['Activity']
    if activity == 'LAYING':
        y_test_transformed.append(0)
    elif activity == 'SITTING':
        y_test_transformed.append(1)
    elif activity == 'WALKING_DOWNSTAIRS':
        y_test_transformed.append(2)
    else:
        y_test_transformed.append(3)
        
y_test_transformed[::100]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 2, 3, 0, 0, 0, 1, 2]

In [13]:
# one sample
data = X_test.iloc[0]
for j, n in enumerate(data):
    input_buffer[j] = n
start_time = time.time()
dma.sendchannel.transfer(input_buffer)
dma.recvchannel.transfer(output_buffer)
dma.sendchannel.wait()
dma.recvchannel.wait()

end_time = time.time()
print(f"Inference time: {(end_time - start_time) * 1000} ms")   

Inference time: 1.7879009246826172 ms


In [ ]:
# whole dataset, record power consumption
recorder = get_recorder()
with recorder.record(0.5):
    
    start_time = time.time()

    num_correct, num_samples = 0, 0
    recorder.mark()
    for i in range(len(X_test)):
        data = X_test.iloc[i]
        for j, n in enumerate(data):
            input_buffer[j] = n
        dma.sendchannel.transfer(input_buffer)
        dma.recvchannel.transfer(output_buffer)
        dma.sendchannel.wait()
        dma.recvchannel.wait()
        num_samples += 1
        num_correct += (y_test_transformed[i] == int(output_buffer[0]))
    recorder.mark()
    end_time = time.time()
    print(f"Accuracy: {(100.0 * num_correct / num_samples):.2f}%")
    print(f"Inference time: {(end_time - start_time):.4f} s")   

In [ ]:
recorder.frame.plot(subplots=True)

In [ ]:
!cpufreq-set -f 200000
!cpufreq-info | grep " current CPU frequency"

In [ ]:
# whole dataset
recorder = get_recorder()

with recorder.record(0.5):
    start_time = time.time()

    num_correct, num_samples = 0, 0
    for i in range(len(X_test)):
        data = X_test.iloc[i]
        for j, n in enumerate(data):
            input_buffer[j] = n
        dma.sendchannel.transfer(input_buffer)
        dma.recvchannel.transfer(output_buffer)
        dma.sendchannel.wait()
        dma.recvchannel.wait()
        num_samples += 1
        num_correct += (y_test_transformed[i] == int(output_buffer[0]))

    end_time = time.time()
    print(f"Accuracy: {(100.0 * num_correct / num_samples):.2f}%")
    print(f"Inference time: {(end_time - start_time):.4f} s") 

In [ ]:
%matplotlib inline
recorder.frame['{}_power'.format(rail_name)].plot()

In [ ]:
# set cpu frequency to default value
!cpufreq-set -f 1200000
!cpufreq-info | grep " current CPU frequency"

In [20]:
# get cpu information
!lscpu | grep CPU*

CPU op-mode(s):                  32-bit, 64-bit
CPU(s):                          4
On-line CPU(s) list:             0-3
CPU max MHz:                     1199.9990
CPU min MHz:                     299.9990


In [21]:
# disable cpu
!echo 0 > /sys/devices/system/cpu/cpu3/online 
!lscpu | grep CPU*

CPU op-mode(s):                  32-bit, 64-bit
CPU(s):                          4
On-line CPU(s) list:             0-2
Off-line CPU(s) list:            3
CPU max MHz:                     1199.9990
CPU min MHz:                     299.9990


In [23]:
# enable disabled cpu
!echo 1 > /sys/devices/system/cpu/cpu3/online 
!lscpu | grep CPU*

CPU op-mode(s):                  32-bit, 64-bit
CPU(s):                          4
On-line CPU(s) list:             0-3
CPU max MHz:                     1199.9990
CPU min MHz:                     299.9990
